In [21]:
import pandas as pd
import numpy as np
import onnxruntime as rt
import matplotlib.pyplot as plt

In [22]:
data = pd.read_csv('../data/all_data.csv')

test_data = data.drop(['Ja', 'Nee'], axis=1)

district_column_names = np.array([s for s in list(data.columns.values) if s.startswith('adres_recentste_wijk_')])

session_model_1 = rt.InferenceSession('../part2_models/model_1.onnx')
session_model_2 = rt.InferenceSession('../part2_models/model_2.onnx')

leeftijd - persoon_leeftijd_bij_onderzoek (18-67)

geslacht - persoon_geslacht_vrouw (binary)

taaleis - persoonlijke_eigenschappen_taaleis_voldaan (binary)

wijk - adres_recentste_wijk_... (binary)


In [40]:
def flip_gender_to_all_men(data):
    transformed_data = data.copy()
    transformed_data['persoon_geslacht_vrouw'] = transformed_data['persoon_geslacht_vrouw'].apply(lambda x: 0 if x == 1 else 0)
    return transformed_data

def flip_gender_to_all_women(data):
    transformed_data = data.copy()
    transformed_data['persoon_geslacht_vrouw'] = transformed_data['persoon_geslacht_vrouw'].apply(lambda x: 1 if x == 0 else 1)
    return transformed_data

def add_children(data, num_children=1):
    transformed_data = data.copy()
    transformed_data['relatie_kind_huidige_aantal'] += num_children
    return transformed_data

def no_children(data):
    transformed_data = data.copy()
    transformed_data['relatie_kind_huidige_aantal'] = 0
    return transformed_data

def modify_migrant_status_all_migrants(data):
    transformed_data = data.copy()
    transformed_data['persoonlijke_eigenschappen_taaleis_voldaan'] = transformed_data['persoonlijke_eigenschappen_taaleis_voldaan'].apply(lambda x: 1 if x == 0 else 1)
    return transformed_data

def modify_migrant_status_no_migrants(data):
    transformed_data = data.copy()
    transformed_data['persoonlijke_eigenschappen_taaleis_voldaan'] = transformed_data['persoonlijke_eigenschappen_taaleis_voldaan'].apply(lambda x: 0 if x == 1 else 0)
    return transformed_data

In [41]:
def metamorphic_test(model_session, data, transform_function, description):
    # Transform the data
    transformed_data = transform_function(data)
    
    # Prepare inputs
    X_original = data.drop(['checked'], axis=1).values.astype(np.float32)
    X_transformed = transformed_data.drop(['checked'], axis=1).values.astype(np.float32)
    
    # Run predictions
    y_pred_original = model_session.run(None, {'input': X_original})[0]
    y_pred_transformed = model_session.run(None, {'input': X_transformed})[0]
    
    # Compare predictions
    differences = np.sum(y_pred_original != y_pred_transformed)
    print(f"{description}: {differences} predictions changed out of {len(y_pred_original)}")

In [ ]:
print("Testing Model 1")
metamorphic_test(session_model_1, test_data, flip_gender_to_all_men, "Gender Flip Test -> All Men")
metamorphic_test(session_model_1, test_data, flip_gender_to_all_women, "Gender Flip Test -> All Women")
metamorphic_test(session_model_1, test_data, no_children, "No Children Test")
metamorphic_test(session_model_1, test_data, lambda x: add_children(x, 2), "Add 2 Children Test")
metamorphic_test(session_model_1, test_data, modify_migrant_status_all_migrants, "Migrant Status Flip Test All Migrants")
metamorphic_test(session_model_1, test_data, modify_migrant_status_no_migrants, "Migrant Status Flip Test No Migrants")

print("\nTesting Model 2")
metamorphic_test(session_model_2, test_data, flip_gender_to_all_men, "Gender Flip Test Men")
metamorphic_test(session_model_2, test_data, flip_gender_to_all_women, "Gender Flip Test Women")
metamorphic_test(session_model_2, test_data, no_children, "No Children Test")
metamorphic_test(session_model_2, test_data, lambda x: add_children(x, 2), "Add 2 Children Test")
metamorphic_test(session_model_2, test_data, modify_migrant_status_all_migrants, "Migrant Status Flip Test All Migrants")
metamorphic_test(session_model_2, test_data, modify_migrant_status_no_migrants, "Migrant Status Flip Test No Migrants")


Testing Model 1
Gender Flip Test Men: 3598 predictions changed out of 130000
Gender Flip Test Women: 7099 predictions changed out of 130000
No Children Test: 4009 predictions changed out of 130000
Add 2 Children Test: 2475 predictions changed out of 130000
Migrant Status Flip Test All Migrants: 1269 predictions changed out of 130000
Migrant Status Flip Test No Migrants: 1699 predictions changed out of 130000

Testing Model 2
Gender Flip Test Men: 587 predictions changed out of 130000
